# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import re
import sys
import os
from sqlalchemy import create_engine
import pickle

#import nltk 
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'wordnet', 'stopwords'])

#import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!ls -ltr

total 56220
-rw-r--r-- 1 root root  1225820 Feb 26  2018 Twitter-sentiment-self-drive-DFE.csv
-rw-r--r-- 1 root root  5064273 Aug 20  2018 messages.csv
-rw-r--r-- 1 root root 11854295 Aug 20  2018 categories.csv
-rw-r--r-- 1 root root     6778 Oct 21 07:01 ETL Pipeline Preparation-zh.ipynb
-rw-r--r-- 1 root root     4693 Oct 21 07:02 ML Pipeline Preparation-zh.ipynb
-rw-r--r-- 1 root root 19599360 Dec  6 12:16 InsertDatabaseName.db
-rw-r--r-- 1 root root  6406144 Dec  6 12:50 DM.db
-rw-r--r-- 1 root root 13197312 Dec  6 15:23 DisasterResponse.db
-rw-r--r-- 1 root root    63215 Dec  6 15:23 ETL Pipeline Preparation.ipynb
-rw-r--r-- 1 root root    34885 Dec  7 11:34 ML Pipeline Preparation.ipynb


In [3]:
# load data from database
engine = create_engine('sqlite:///DM.db')
df = pd.read_sql_table('DM', con=engine)
categories = df.columns[4:]

X = df['message']
y = df.iloc[:, 4:]

In [4]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    # Detecte URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    words = word_tokenize(text)
    
    # remove stop words
    stopwords_ = stopwords.words("english")
    words = [word for word in words if word not in stopwords_]
    
    # extract root form of words
    words = [WordNetLemmatizer().lemmatize(word, pos='v') for word in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
category_names = y.columns
for i, column in enumerate(category_names):
        print(classification_report(y_test[column], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.62      0.48      0.54      1509
          1       0.85      0.91      0.88      5004
          2       0.23      0.41      0.29        41

avg / total       0.80      0.80      0.80      6554

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5428
          1       0.81      0.41      0.54      1126

avg / total       0.87      0.88      0.86      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

             precision    recall  f1-score   support

          0       0.75      0.87      0.80      3836
          1       0.75      0.59      0.66      2718

avg / total       0.75      0.75      0.74      6554

             precision    recall  f1-score   support

          0       0.93      1.00 

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {'clf__estimator__n_estimators': [20, 50],}

cv =   GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3, n_jobs=-1)

In [13]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................
[CV]  clf__estimator__n_estimators=20, score=0.2491608178211779, total= 1.4min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.2537381751602075, total= 1.4min
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, score=0.24351541043637473, total= 1.4min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.2657918828196521, total= 3.2min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.259231003967043, total= 3.2min
[CV] clf__estimator__n_estimators=50 .................................
[CV]  clf__estimator__n_estimators=50, score=0.25663716814159293, total= 3.1min


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 15.8min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [14]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [15]:
cv.best_params_

{'clf__estimator__n_estimators': 50}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
y_pred = cv.predict(X_test)

In [17]:
for i, column in enumerate(category_names):
        print(classification_report(y_test[column], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.68      0.42      0.52      1509
          1       0.84      0.93      0.89      5004
          2       0.24      0.46      0.32        41

avg / total       0.80      0.81      0.80      6554

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5428
          1       0.85      0.48      0.62      1126

avg / total       0.89      0.90      0.88      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

             precision    recall  f1-score   support

          0       0.79      0.85      0.82      3836
          1       0.76      0.67      0.71      2718

avg / total       0.77      0.78      0.77      6554

             precision    recall  f1-score   support

          0       0.93      1.00 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Linear Support Vector Classification

In [18]:

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(
                            OneVsRestClassifier(LinearSVC())))])

    # hyper-parameter grid
parameters = {'vect__ngram_range': ((1, 1), (1, 2)),
                  'vect__max_df': (0.75, 1.0)
                  }

    # create model
model = GridSearchCV(estimator=pipeline,
            param_grid=parameters,
            verbose=3,
            cv=3)

In [19]:
model.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.27799816905706437, total=  15.0s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.7s remaining:    0.0s


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.2831858407079646, total=  15.1s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   47.4s remaining:    0.0s


[CV]  vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.2705218187366494, total=  14.9s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.28501678364357647, total=  18.9s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.2854745193774794, total=  18.4s
[CV] vect__max_df=0.75, vect__ngram_range=(1, 2) .....................
[CV]  vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.27418370460787306, total=  18.3s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.27799816905706437, total=  15.0s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.2831858407079646, total=  14.9s
[CV] vect__max_df=1.0, vect__ngram_range=(1, 1) ......................
[CV]  vect__max_df=1.0, vec

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  5.1min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [20]:
y_pred = model.predict(X_test)

In [21]:
for i, column in enumerate(category_names):
        print(classification_report(y_test[column], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.69      0.48      0.57      1509
          1       0.85      0.93      0.89      5004
          2       0.47      0.20      0.28        41

avg / total       0.81      0.83      0.81      6554

             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5428
          1       0.75      0.62      0.68      1126

avg / total       0.89      0.90      0.90      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

             precision    recall  f1-score   support

          0       0.82      0.77      0.80      3836
          1       0.70      0.77      0.73      2718

avg / total       0.77      0.77      0.77      6554

             precision    recall  f1-score   support

          0       0.95      0.98 

### 9. Export your model as a pickle file

In [25]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [26]:
!jupyter nbconvert --to script *.ipynb

[NbConvertApp] Converting notebook ETL Pipeline Preparation-zh.ipynb to script
[NbConvertApp] Writing 2765 bytes to ETL Pipeline Preparation-zh.py
[NbConvertApp] Converting notebook ETL Pipeline Preparation.ipynb to script
[NbConvertApp] Writing 4839 bytes to ETL Pipeline Preparation.py
[NbConvertApp] Converting notebook ML Pipeline Preparation-zh.ipynb to script
[NbConvertApp] Writing 1336 bytes to ML Pipeline Preparation-zh.py
[NbConvertApp] Converting notebook ML Pipeline Preparation.ipynb to script
[NbConvertApp] Writing 5963 bytes to ML Pipeline Preparation.py


In [27]:
ls

categories.csv                     InsertDatabaseName.db
cv.pkl                             messages.csv
DisasterResponse.db                ML Pipeline Preparation.ipynb
DM.db                              ML Pipeline Preparation.py
ETL Pipeline Preparation.ipynb     ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation.py        ML Pipeline Preparation-zh.py
ETL Pipeline Preparation-zh.ipynb  model.pkl
ETL Pipeline Preparation-zh.py     Twitter-sentiment-self-drive-DFE.csv
